# Comparing Site Definitions: Basic Ion Migration in a FCC Lattice

## Introduction

This tutorial demonstrates how to track ion migration through a face-centered cubic (FCC) lattice using three different approaches for site definition:

1. **Spherical Sites**: Simple spheres centered at interstitial positions
2. **Voronoi Sites**: Space-filling cells based on proximity to site centers
3. **Polyhedral Sites**: Sites defined by coordination polyhedra

We will analyse a simple trajectory where a single lithium ion follows a curved path from an octahedral site, through a tetrahedral site, to another octahedral site in an fcc oxygen lattice. This trajectory mimics a common diffusion mechanism in close-packed structures.

This tutorial illustrates:
- How to set up different site definitions using the TrajectoryBuilder
- The tradeoffs between different site definition approaches
- How site definitions affect the analysis of ion migration paths

In [6]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from pymatgen.core import Lattice, Structure, DummySpecies
from site_analysis import TrajectoryBuilder
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator

## Creating a FCC Lattice

First, we create a 3×3×3 fcc oxygen lattice, which will serve as our host framework. 

In [5]:
a = 4.0 # lattice parameter
lattice = Lattice.cubic(a)
supercell_expansion = [3,3,3]
fcc_structure = Structure.from_spacegroup(
        sg='Fm-3m',
        lattice=lattice,
        species=['O'],
        coords=[[0, 0, 0]]
    ) * supercell_expansion

Full Formula (O108)
Reduced Formula: O2
abc   :  12.000000  12.000000  12.000000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (108)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  O     0         0         0
  1  O     0         0         0.333333
  2  O     0         0         0.666667
  3  O     0         0.333333  0
  4  O     0         0.333333  0.333333
  5  O     0         0.333333  0.666667
  6  O     0         0.666667  0
  7  O     0         0.666667  0.333333
  8  O     0         0.666667  0.666667
  9  O     0.333333  0         0
 10  O     0.333333  0         0.333333
 11  O     0.333333  0         0.666667
 12  O     0.333333  0.333333  0
 13  O     0.333333  0.333333  0.333333
 14  O     0.333333  0.333333  0.666667
 15  O     0.333333  0.666667  0
 16  O     0.333333  0.666667  0.333333
 17  O     0.333333  0.666667  0.666667
 18  O     0.666667  0         0
 19  O     0.666667  0         0.3

The fcc structure has two types of interstitial sites:

- **Octahedral sites**: Located at the edge midpoints (0.5, 0.0, 0.0) and the body center (0.5, 0.5, 0.5)
- **Tetrahedral sites**: Located at positions like (0.25, 0.25, 0.25), (0.75, 0.25, 0.25), etc.

In [22]:
fcc_octahedral_coords = (Structure.from_spacegroup(
    sg='Fm-3m',
    lattice=lattice,
    species=[DummySpecies('Q')],
    coords=[[0.5, 0, 0]]
) * supercell_expansion).frac_coords

fcc_tetrahedral_coords = (Structure.from_spacegroup(
    sg='Fm-3m',
    lattice=lattice,
    species=[DummySpecies('Q')],
    coords=[[0.25, 0.25, 0.25]]
) * supercell_expansion).frac_coords

## Generating a Mobile Ion Trajectory

Next, we'll create a trajectory with a lithium ion moving along a curved path from an octahedral site, through a tetrahedral site, to another octahedral site. This simulates a typical diffusion mechanism in an FCC lattice.

In [38]:
mobile_ion_coords = np.array(
    [[0.3333    , 0.5       , 0.5       ],
     [0.3499784 , 0.5299988 , 0.5166612 ],
     [0.36665493, 0.55333121, 0.53332214],
     [0.38332959, 0.56999721, 0.54998281],
     [0.40000239, 0.57999681, 0.56664321],
     [0.41667332, 0.58333001, 0.58330334],
     [0.43334239, 0.57999681, 0.59996321],
     [0.45000959, 0.56999721, 0.61662281],
     [0.46667493, 0.55333121, 0.63328214],
     [0.4833384 , 0.5299988 , 0.6499412 ],
     [0.5       , 0.5       , 0.6666    ]])

In [41]:
md_trajectory = []
for c in mobile_ion_coords:
    structure = fcc_structure.copy().append(species='Li', coords=c)
    md_trajectory.append(structure)

## Analyzing with Spherical Sites

First, we'll analyze the trajectory using spherical sites. We'll define spherical sites at both the octahedral and tetrahedral positions.

In [48]:
fcc_octahedral_coords

array([[0.16666667, 0.        , 0.        ],
       [0.16666667, 0.        , 0.33333333],
       [0.16666667, 0.        , 0.66666667],
       [0.16666667, 0.33333333, 0.        ],
       [0.16666667, 0.33333333, 0.33333333],
       [0.16666667, 0.33333333, 0.66666667],
       [0.16666667, 0.66666667, 0.        ],
       [0.16666667, 0.66666667, 0.33333333],
       [0.16666667, 0.66666667, 0.66666667],
       [0.5       , 0.        , 0.        ],
       [0.5       , 0.        , 0.33333333],
       [0.5       , 0.        , 0.66666667],
       [0.5       , 0.33333333, 0.        ],
       [0.5       , 0.33333333, 0.33333333],
       [0.5       , 0.33333333, 0.66666667],
       [0.5       , 0.66666667, 0.        ],
       [0.5       , 0.66666667, 0.33333333],
       [0.5       , 0.66666667, 0.66666667],
       [0.83333333, 0.        , 0.        ],
       [0.83333333, 0.        , 0.33333333],
       [0.83333333, 0.        , 0.66666667],
       [0.83333333, 0.33333333, 0.        ],
       [0.

In [47]:

# Using the builder pattern to create a trajectory for analysis
builder = TrajectoryBuilder()

# Set the structure and mobile species
builder.with_structure(md_trajectory[0])
builder.with_mobile_species("Li")

# In this example we choose sphere radii corresponding to the inradii of the fcc octahedra and tetrahedra.
# These are the spheres that touch the polyhedral faces: https://en.wikipedia.org/wiki/Inscribed_sphere
# r_oct = l / sqrt(6) = 1.155 Å
# r_tet = l x sqrt(6) / 12 = 0.577 Å

# Define spherical sites at octahedral positions
builder.with_spherical_sites(
    centres=fcc_octahedral_coords,
    radii=1.155,
    labels="octahedral"
)

# Define spherical sites at tetrahedral positions
builder.with_spherical_sites(
    centres=fcc_tetrahedral_coords,
    radii=0.577,
    labels="tetrahedral"
)

# Build the trajectory
trajectory = builder.build()

# Analyze all structures in the trajectory
trajectory.trajectory_from_structures(md_trajectory, progress=True)

spherical_trajectory = trajectory.atoms[0].trajectory
spherical_trajectory

100%|█████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 137.31 steps/s]


[None, None, None, None, 283, 283, 283, None, None, None, None]

In [53]:
trajectory.sites[283]

## Analyzing with Voronoi Sites

Next, we'll analyze the same trajectory using Voronoi sites. Voronoi sites partition space based on proximity to site centers, with no gaps or overlaps.

In [ ]:
def analyze_with_voronoi_sites(trajectory_structures, octahedral_sites, tetrahedral_sites):
    """
    Analyze the trajectory using Voronoi sites.
    
    Args:
        trajectory_structures: List of structures with the mobile ion
        octahedral_sites: List of octahedral site positions
        tetrahedral_sites: List of tetrahedral site positions
        
    Returns:
        Trajectory object with the analysis results
    """
    # Using the builder pattern to create a trajectory for analysis
    builder = TrajectoryBuilder()
    
    # Set the structure and mobile species
    builder.with_structure(trajectory_structures[0])
    builder.with_mobile_species("Li")
    
    # Define Voronoi sites at octahedral positions
    builder.with_voronoi_sites(
        centres=octahedral_sites,
        labels="octahedral"
    )
    
    # Define Voronoi sites at tetrahedral positions
    builder.with_voronoi_sites(
        centres=tetrahedral_sites,
        labels="tetrahedral"
    )
    
    # Build the trajectory
    trajectory = builder.build()
    
    # Analyze all structures in the trajectory
    trajectory.trajectory_from_structures(trajectory_structures, progress=True)
    
    return trajectory

# Analyze with Voronoi sites
print("Analyzing with Voronoi sites...")
voronoi_trajectory = analyze_with_voronoi_sites(trajectory_structures, octahedral_sites, tetrahedral_sites)

## Analyzing with Polyhedral Sites

Finally, we'll analyze the trajectory using polyhedral sites. Polyhedral sites are defined by coordination polyhedra formed by the host lattice atoms.

In [ ]:
def create_reference_structure_with_sites(octahedral_sites, tetrahedral_sites):
    """
    Create a reference structure with dummy atoms at the octahedral and tetrahedral sites.
    This reference structure will be used to define polyhedral sites.
    
    Args:
        octahedral_sites: List of octahedral site positions
        tetrahedral_sites: List of tetrahedral site positions
        
    Returns:
        A pymatgen Structure object with dummy atoms at interstitial sites
    """
    # Create a copy of the FCC lattice
    reference = fcc_lattice.copy()
    
    # Add dummy atoms at octahedral sites (use Mg for octahedral sites)
    for site in octahedral_sites:
        reference.append("Mg", site)
    
    # Add dummy atoms at tetrahedral sites (use Na for tetrahedral sites)
    for site in tetrahedral_sites:
        reference.append("Na", site)
    
    return reference

def analyze_with_polyhedral_sites(trajectory_structures, octahedral_sites, tetrahedral_sites):
    """
    Analyze the trajectory using polyhedral sites.
    
    Args:
        trajectory_structures: List of structures with the mobile ion
        octahedral_sites: List of octahedral site positions
        tetrahedral_sites: List of tetrahedral site positions
        
    Returns:
        Trajectory object with the analysis results
    """
    # Create a reference structure with dummy atoms at interstitial sites
    reference_structure = create_reference_structure_with_sites(octahedral_sites, tetrahedral_sites)
    
    # Using the builder pattern to create a trajectory for analysis
    builder = TrajectoryBuilder()
    
    # Set the structure, reference structure, and mobile species
    builder.with_structure(trajectory_structures[0])
    builder.with_reference_structure(reference_structure)
    builder.with_mobile_species("Li")
    
    # Configure alignment between reference and target structures
    builder.with_structure_alignment(align_species="Cu")
    
    # Define polyhedral sites at octahedral positions using Mg dummy atoms
    builder.with_polyhedral_sites(
        centre_species="Mg",  # Mg represents octahedral sites in our reference structure
        vertex_species="Cu",  # Host lattice Cu atoms form the vertices
        cutoff=3.0,          # Distance cutoff for finding vertices
        n_vertices=6,        # Octahedral coordination has 6 vertices
        label="octahedral"
    )
    
    # Define polyhedral sites at tetrahedral positions using Na dummy atoms
    builder.with_polyhedral_sites(
        centre_species="Na",  # Na represents tetrahedral sites in our reference structure
        vertex_species="Cu",  # Host lattice Cu atoms form the vertices
        cutoff=2.5,          # Distance cutoff for finding vertices
        n_vertices=4,        # Tetrahedral coordination has 4 vertices
        label="tetrahedral"
    )
    
    # Build the trajectory
    trajectory = builder.build()
    
    # Analyze all structures in the trajectory
    trajectory.trajectory_from_structures(trajectory_structures, progress=True)
    
    return trajectory

# Analyze with polyhedral sites
print("Analyzing with polyhedral sites...")
polyhedral_trajectory = analyze_with_polyhedral_sites(trajectory_structures, octahedral_sites, tetrahedral_sites)

## Comparing the Results

Now we'll compare how the different site definitions assigned the mobile lithium ion throughout its trajectory.

In [ ]:
def get_site_types(trajectory):
    """
    Get the site type assignments for each timestep.
    
    Args:
        trajectory: A Trajectory object with analysis results
        
    Returns:
        List of site types ('octahedral', 'tetrahedral', or None) for each timestep
    """
    # Get the site indices from the first (and only) atom's trajectory
    atom = trajectory.atoms[0]
    site_indices = atom.trajectory
    
    # Convert site indices to site labels
    site_types = []
    for site_idx in site_indices:
        if site_idx is None:
            site_types.append(None)
        else:
            site = trajectory.sites[site_idx]
            site_types.append(site.label)
    
    return site_types

def plot_site_assignments(trajectories, labels):
    """
    Plot the site assignments for each analysis method.
    
    Args:
        trajectories: List of Trajectory objects from different analysis methods
        labels: List of labels for each analysis method
    """
    # Get site types for each trajectory
    all_site_types = []
    for trajectory in trajectories:
        all_site_types.append(get_site_types(trajectory))
    
    # Convert site types to numerical values for plotting
    site_type_values = []
    for site_types in all_site_types:
        values = []
        for site_type in site_types:
            if site_type == "octahedral":
                values.append(1)
            elif site_type == "tetrahedral":
                values.append(0)
            else:  # None (no assigned site)
                values.append(-1)
        site_type_values.append(values)
    
    # Create the plot
    fig, axes = plt.subplots(len(trajectories), 1, figsize=(10, 8), sharex=True)
    
    # Handle single trajectory case
    if len(trajectories) == 1:
        axes = [axes]
    
    for i, (values, ax, label) in enumerate(zip(site_type_values, axes, labels)):
        # Get the timesteps
        timesteps = range(len(values))
        
        # Create a step plot
        ax.step(timesteps, values, where='mid', linewidth=2, label=label)
        
        # Set y-axis
        ax.set_yticks([-1, 0, 1])
        ax.set_yticklabels(['None', 'Tetrahedral', 'Octahedral'])
        ax.set_ylim(-1.5, 1.5)
        ax.grid(True, linestyle='--', alpha=0.7)
        
        # Add a title
        ax.set_title(f"{label} Sites")
    
    # Set x-axis label on the bottom plot
    axes[-1].set_xlabel('Timestep')
    
    plt.tight_layout()
    plt.show()

# Compare the results
plot_site_assignments(
    [spherical_trajectory, voronoi_trajectory, polyhedral_trajectory],
    ["Spherical", "Voronoi", "Polyhedral"]
)

## Analysis and Discussion

Let's analyze the differences in site assignments between the three approaches:

### Spherical Sites
- **Advantages**: Simple to implement and understand
- **Limitations**: Can leave gaps between sites or create overlapping regions
- **Behavior in our example**: May show unassigned regions (None) where the ion is between sites

### Voronoi Sites
- **Advantages**: Completely fills space with no gaps or overlaps
- **Limitations**: Site boundaries are determined solely by proximity, not by physical barriers
- **Behavior in our example**: Always assigns the ion to either octahedral or tetrahedral sites

### Polyhedral Sites
- **Advantages**: Physically meaningful site definitions based on coordination polyhedra
- **Limitations**: More complex to set up and may not cover all space
- **Behavior in our example**: Most accurate representation of actual tetrahedral and octahedral sites

The differences in site assignment are most apparent in transition regions, where the ion is moving between sites. The choice of site definition can significantly impact the analysis of diffusion mechanisms, especially the timing of transitions between sites.

## Conclusion

This tutorial has demonstrated how to set up and compare different site definitions for analyzing ion migration in a simple FCC lattice. The key takeaways are:

1. Different site definitions yield different site assignments, especially during transitions
2. The choice of site definition should be based on the specific research question and material system
3. For simple systems, Voronoi sites provide complete spatial coverage with minimal setup
4. For more accurate physical representations, polyhedral sites can better capture the actual coordination environments

In the next tutorial, we'll apply these concepts to the more complex argyrodite structure, where multiple distinct tetrahedral site types and chemical disorder add further complexity to the analysis.